# Modifying pretrained model

In [43]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2, numpy as np

from keras import backend as K

K.set_image_dim_ordering('th')

model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
model.add(Conv2D(64, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
'''
model.add(Conv2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(64, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
'''
model.add(Conv2D(32, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))

model.add(Conv2D(2, 3, 3, activation='softmax'))
#model.add(Activation(2, 1, 1, activation='sigmoid'))
#model.add(Conv2D(2, 1, 1, activation='sigmoid'))
model.summary()

# https://github.com/jocicmarko/ultrasound-nerve-segmentation/blob/master/train.py#L19
smooth = 1.
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['categorical_crossentropy','accuracy'])
          #loss=dice_coef_loss, metrics=[dice_coef,'accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_16 (ZeroPaddi (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_17 (ZeroPaddi (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 32, 224, 224)      18464     
_________________________________________________________________
zero_padding2d_18 (ZeroPaddi (None, 32, 226, 226)      0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 2, 224, 224)       578       
Total params: 20,834
Trainable params: 20,834
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(2, (3, 3), activation="softmax")`


In [ ]:
model.fit(x_train,y_train,epochs=500,verbose=2)
#m.fit(preprocess(mri_data[:,:,20]),preprocess(labelled_data[:,:,20], False),epochs=20)

Epoch 1/500
0s - loss: 1043.5720 - categorical_crossentropy: 1043.5721 - acc: 0.0848
Epoch 2/500
0s - loss: 1480.8684 - categorical_crossentropy: 1480.8684 - acc: 0.0012
Epoch 3/500
0s - loss: 1412.0901 - categorical_crossentropy: 1412.0901 - acc: 4.9603e-04
Epoch 4/500
0s - loss: 736.1298 - categorical_crossentropy: 736.1297 - acc: 0.0012
Epoch 5/500
0s - loss: 1053.3611 - categorical_crossentropy: 1053.3612 - acc: 0.2418
Epoch 6/500
0s - loss: 785.9198 - categorical_crossentropy: 785.9199 - acc: 0.0565
Epoch 7/500
0s - loss: 963.2289 - categorical_crossentropy: 963.2291 - acc: 0.0025
Epoch 8/500
0s - loss: 772.1252 - categorical_crossentropy: 772.1253 - acc: 0.0856
Epoch 9/500
0s - loss: 619.9008 - categorical_crossentropy: 619.9007 - acc: 7.4405e-04
Epoch 10/500
0s - loss: 616.2150 - categorical_crossentropy: 616.2149 - acc: 4.9603e-04
Epoch 11/500
0s - loss: 618.7922 - categorical_crossentropy: 618.7922 - acc: 9.9206e-04
Epoch 12/500
0s - loss: 644.1367 - categorical_crossentropy: 

In [3]:
# Get image data w/ nibabel
import nibabel as nib
def get_data(filename):
    img = nib.load(filename)
    return img.get_data()
def scale(array):
    return (array-np.min(array))/(np.max(array) - np.min(array))    
mri_data = scale(get_data('../data/case_001_2.nii.gz')).transpose((2,0,1))
labelled_data = get_data('../data/case_001_labels.nii.gz').transpose((2,0,1))

In [4]:
from scipy.misc import imresize

def preprocess(mri_slice,tile=True):
    img0 = imresize(mri_slice, (224,224))
    img0.resize((img0.shape[0],img0.shape[1],1))
    if tile: img0 = np.tile(img0,3)
    img0 = img0.transpose((2,0,1))
    return np.expand_dims(img0, axis=0)
def preprocess_label(labels):
    labels = imresize(labels, (224,224))
    pos = np.where(labels==1)
    new_labels = np.zeros([labels.shape[0],labels.shape[1],2])
    new_labels[:,:,0] = 1
    new_labels[pos[0],pos[1],0] = 0
    new_labels[pos[0],pos[1],1] = 1
    new_labels = new_labels.transpose((2,0,1))
    return np.expand_dims(new_labels, axis=0)


In [74]:
dice_coef(y_train[0],y_train[0])

<tf.Tensor 'truediv_21:0' shape=() dtype=float64>

In [6]:
label_sums = labelled_data.sum(axis=(1,2))
x_train_arr = []
y_train_arr = [] 
#for mri_slice, labels in zip(mri_data, labelled_data):
for mri_slice, labels in zip(mri_data[np.where(label_sums>0)], labelled_data[np.where(label_sums>0)]):
    x_train_arr.append(preprocess(mri_slice))
    y_train_arr.append(preprocess_label(labels))
x_train = np.vstack(x_train_arr)
y_train = np.vstack(y_train_arr)

In [ ]:
np.histogram(img,bins=20)

In [15]:
#img= model.predict(preprocess(mri_data[24])).squeeze()

#img[np.argmax(img, axis=0)]
pos = np.where(np.argmax(img, axis=0)>0)
img0 = np.zeros((img.shape[1],img.shape[2]))
img0[pos[0],pos[1]] = 1

0.0

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

predict_slide = 21
img= model.predict(preprocess(mri_data[predict_slide])).squeeze()
pos = np.where(np.argmax(img, axis=0)>0)
img1 = np.zeros((img.shape[1],img.shape[2]))
img1[pos[0],pos[1]] = 1
fig, ax = plt.subplots(1,1)
#img1 = np.copy(img)
#img1[img<1] = 0
#ax.imshow(new_data)
#ax.imshow(imresize(mri_data[predict_slide], (224,224)),cmap=plt.cm.gray)
ax.imshow(np.ma.masked_where(img1<1, img1),alpha=.7,cmap=plt.cm.autumn, interpolation='none')
#plt.imshow(img1)
#actual = imresize(mri_data[predict_slide], (224,224))

#ax.imshow(np.ma.masked_where(actual==0, actual),alpha=.5,cmap=plt.cm.autumn, interpolation='none')
#plt.imshow(np.squeeze(img))

In [ ]:
img  
np.where((img>.5))[0]

In [ ]:
plt.imshow(preprocess(labelled_data[22], False).squeeze())
#plt.imshow(labelled_data[22])
labelled_data[20].sum()

In [ ]:

label_sums = labelled_data.sum(axis=(1,2))
train_data = mri_data[np.where(label_sums==1)]
for i in range(0, mri_data.shape[2]):
    if skip_empty_slices:
        if np.sum(labelled_data[:,:,i]) == 0: continue
    train.append[i]
md = imresize(mri_data.transpose((2,0,1)), (33,224,224))
md.shape

In [ ]:
# Get image data w/ nibabel
import nibabel as nib
def get_data(filename):
    img = nib.load(filename)
    return img.get_data()
def scale(array):
    return (array-np.min(array))/(np.max(array) - np.min(array))
mri_data = scale(get_data('../data/case_001_2.nii.gz'))
# Generate input to model (shape=(1,3,224,224))
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.misc import imresize
#plt.tight_layout()
img0 = imresize(mri_data[:,:,24], (224,224))
img = plt.imshow(img0) 

img0.resize((img0.shape[0],img0.shape[1],1))
img0_rgb = np.tile(img0,3)
img0_rgb = img0_rgb.transpose((2,0,1))
img0_rgb = np.expand_dims(img0_rgb, axis=0)
img0_rgb.shape

In [ ]:
print(model.summary())

In [ ]:
from keras.models import Model

def get_intermediate(layer_name,inp):
    intermediate_layer_model = Model(inputs=model.input,
                                     outputs=model.get_layer(layer_name).output)
    return np.squeeze(intermediate_layer_model.predict(inp))
import math
def show_range(data, plot_range):
    #plot_range = range(0,derp.shape[0],10)
    i = 0
    
    f, axarr = plt.subplots(math.ceil(len(plot_range)/3), 3)
    f.set_size_inches([10,10])
    for i in range(len(plot_range)):
        if len(axarr.shape)==1:
            ax = axarr[i]
        else:
            ax = axarr[int(i/3),i%3]
        ax.imshow(data[plot_range[i]])
        ax.set_title("Node %d"%plot_range[i])
    plt.show()

In [ ]:
layer_name = 'conv2d_2'
out_conv2d_2 = get_intermediate(layer_name,img0_rgb)
show_range(out_conv2d_2, range(1,out_conv2d_2.shape[0],10))

In [ ]:
import keras
keras.backend.image_data_format()